# HW 7
# FINM 36700
Group C 26

### Functions used

In [6]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import seaborn as sns
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [7]:
def performance_summary(return_data, annualization = 12):
    """
        Inputs: 
            return_data - DataFrame with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly returns. 
    """
    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*annualization)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(annualization))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']

    summary_stats['Min Value'] = return_data.min()
    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats

In [8]:
def regression_based_performance(factor,fund_ret,intercept = True):   # inputs: (x, y(usually equity returns), rf)
    """ 
        Returns the Regression based performance Stats for given set of returns and factors
        Output:
            summary_stats - (Beta of regression, treynor ratio, information ratio, alpha). 
    """
    if intercept:
        X = sm.tools.add_constant(factor)
    else:
        X = factor
    y=fund_ret
    model = sm.OLS(y,X,missing='drop').fit()
    
    if intercept:
        beta = model.params[1:]
        alpha = round(float(model.params['const']),6)*12
        
    else:
        beta = model.params
    treynor_ratio = ((fund_ret.values).mean()*12)/beta[0]
    tracking_error = (model.resid.std()*np.sqrt(12))
    if intercept:        
        information_ratio = model.params[0]*12/tracking_error
    r_squared = model.rsquared
    if intercept:
        return (beta,treynor_ratio,information_ratio,alpha,r_squared,tracking_error)
    else:
        return (beta,treynor_ratio,r_squared,tracking_error)

In [9]:
def rolling_regression_param(factor,fund_ret,roll_window = 60):
    """ 
        Returns the Rolling Regression parameters for given set of returns and factors
        Inputs:
            factor - Dataframe containing monthly returns of the regressors
            fund_ret - Dataframe containing monthly excess returns of the regressand fund
            roll_window = rolling window for regression
        Output:
            params - Dataframe with time-t as the index and constant and Betas as columns
    """
    X = sm.add_constant(factor)
    y= fund_ret
    rols = RollingOLS(y, X, window=roll_window)
    rres = rols.fit()
    params = rres.params.copy()
    params.index = np.arange(1, params.shape[0] + 1)
    return params

### Data Input

In [10]:
gmo_data = pd.read_excel(r'gmo_analysis_data.xlsx', sheet_name='returns (total)',index_col = 0)
rf_data = pd.read_excel(r'gmo_analysis_data.xlsx', sheet_name='risk-free rate',index_col = 0)
gmo = pd.DataFrame(gmo_data)
rf = pd.DataFrame(rf_data)
gmo['SPY'] = gmo['SPY']-rf_data['US3M']
gmo['GMWAX'] = gmo['GMWAX'] - rf_data['US3M']
gmo

,SPY,GMWAX
1993-02-28,0.008159,NaN
1993-03-31,0.019949,NaN
1993-04-30,-0.028064,NaN
1993-05-31,0.024361,NaN
1993-06-30,0.001084,NaN
...,...,...
2023-06-30,0.060289,0.035234
2023-07-31,0.028108,0.019797
2023-08-31,-0.020885,-0.025650
2023-09-30,-0.052018,-0.020966


In [11]:
signals_data = pd.read_excel(r'gmo_analysis_data.xlsx', sheet_name='signals',index_col = 0)
signals = pd.DataFrame(signals_data)
signals.tail(5)

,DP,EP,US10Y
2023-06-30,1.58,3.88,3.81
2023-07-31,1.53,3.76,3.97
2023-08-31,1.55,3.89,4.09
2023-09-30,1.57,4.11,4.59
2023-10-31,1.62,4.18,4.88


# 2. Analyzing GMO

### 2.1 Subsamples

In [12]:
sub_samples = {
              '1993-2011' : ['1993','2011'],
              '2012-2023' : ['2012','2023'],
              '1993-2023' : ['1993','2023'],
              }

gmo_sum = []
for k,v in sub_samples.items():
    sub_gmo = gmo.loc[sub_samples[k][0]:sub_samples[k][1],['GMWAX']].dropna()
    gmo_summary = performance_summary(sub_gmo)
    gmo_summary = gmo_summary
    gmo_summary.index = [k]
    gmo_sum.append(gmo_summary)

gmo_summary = pd.concat(gmo_sum)
gmo_summary.loc[:,['Mean','Volatility','Sharpe Ratio']]

,Mean,Volatility,Sharpe Ratio
1993-2011,0.015827,0.125011,0.126603
2012-2023,0.036436,0.094503,0.385556
1993-2023,0.024859,0.112537,0.220898


The mean has increased from 2012 to 2023 compared with the previous period (from 1993 to 2011) while the volatility has decreased. The performance of GMWAX has improved recently and the Sharpe Ratio has become higher in the recent period despite pandemics and other financial difficulties. 

### 2.2 Extreme Scenarios

In [13]:
tail_risk_summary = gmo_summary.loc[:,['Min Value', 'VaR (0.05)','Max Drawdown']]
tail_risk_summary

,Min Value,VaR (0.05),Max Drawdown
1993-2011,-0.149179,-0.059806,-0.472946
2012-2023,-0.118650,-0.037826,-0.226046
1993-2023,-0.149179,-0.047061,-0.472946


(a) The minimum return, VaR, and max drawdown are kept within reasonable ranges; GMWAX has a low tail-risk given these statistics.  

(b) The risk management performance of GMWAX improved from 2012 to 2023 given better minimum return, smaller VaR, and smaller max drawdown compared with the previous period. The improvement is obvious and it is reasonable to believe that GMWAX gains better predictive skills in its asset allocation strategy, but it should be cautious that the latter period has a shorter time horizon. 

### 2.3 Regression

In [14]:
reg_sub_sample = []
for k,v in sub_samples.items():    
    y = gmo.loc[sub_samples[k][0]:sub_samples[k][1],['GMWAX']].dropna()
    X = gmo.loc[y.index[0]:y.index[-1],['SPY']]
    reg = regression_based_performance(X,y)
    beta_mkt = reg[0][0]
    alpha = reg[3]
    r_squared = reg[4]
    reg_sub_sample.append(pd.DataFrame([[beta_mkt,alpha,r_squared]],columns=['SPY Beta','Alpha','R-Squared'],index = [k]))

reg_performance = pd.concat(reg_sub_sample)
reg_performance

,SPY Beta,Alpha,R-Squared
1993-2011,0.539615,-0.005748,0.507129
2012-2023,0.573764,-0.032652,0.754377
1993-2023,0.550609,-0.016572,0.582145


(b) The SPY Beta is 0.54 in the 1993 - 2011 subsample and 0.57 in the 2012 - 2023 subsample; overall, the beta value is 0.55. It is a moderator beta and it is valid to say GMWAX practices low-beta strategy. The SPY beta does not vary significantly in the past periods. 

(c) Both subsample shows negative alpha values; the alpha from 2012 - 2023 is -0.03 while the value is -0.006 in the previous period. The alpha value becomes more negative in the recent period, which indicates a slightly bad message in investment returns if considering SPY as the benchmark. 

# 3. Forecast Regression

### 让weight based on forecast：如果forecast return twice as today, weight double；如果forecast return decrease 80%, weight decrease 80%

### 3.1 Lagged Regression

In [15]:
gmo_total = pd.DataFrame(gmo_data)

#### DP_single

In [16]:
y = gmo_total[['SPY']]
X = signals[['DP']].shift(1)
reg = regression_based_performance(X,y)
beta_dp = reg[0][0]
alpha = reg[3]
r_squared = reg[4]
dp_forecast = pd.DataFrame([[beta_dp,alpha,r_squared]],columns=['DP Beta','Alpha','R-Squared'],index = ['DP'])
dp_forecast


,DP Beta,Alpha,R-Squared
DP,0.009516,-0.113772,0.009359


#### EP_single

In [17]:
y = gmo_total[['SPY']]
X = signals[['EP']].shift(1)
reg = regression_based_performance(X,y)
beta_ep = reg[0][0]
alpha = reg[3]
r_squared = reg[4]
ep_forecast = pd.DataFrame([[beta_ep,alpha,r_squared]],columns=['EP Beta','Alpha','R-Squared'],index = ['EP'])
ep_forecast

,EP Beta,Alpha,R-Squared
EP,0.003252,-0.073932,0.008692


#### DP,EP,RF

In [18]:
y = gmo_total[['SPY']]
X = signals[['DP', 'EP','US10Y']].shift(1)
reg = regression_based_performance(X,y)
beta_dp = reg [0][0]
beta_ep = reg[0][1]
beta_rf = reg[0][2]
alpha = reg[3]
r_squared = reg[4]
forecast = pd.DataFrame([[beta_dp,beta_ep,beta_rf,alpha,r_squared]],columns=['DP Beta','EP Beta','RF Beta','Alpha','R-Squared'],index = ['3_factors'])
forecast

,DP Beta,EP Beta,RF Beta,Alpha,R-Squared
3_factors,0.008023,0.002694,-0.000982,-0.180768,0.016364


#### R_Squared

In [19]:
dp_rsq = dp_forecast.loc['DP','R-Squared']
ep_rsq = ep_forecast.loc['EP','R-Squared']
factors_rsq = forecast.loc['3_factors','R-Squared']
R_Squared = pd.DataFrame({'R_Squared':[dp_rsq,ep_rsq,factors_rsq]}, index = ["DP","EP","3_regressors"])
R_Squared

,R_Squared
DP,0.009359
EP,0.008692
3_regressors,0.016364


### 3.2 Strategies

In [20]:
dp_return = (dp_forecast['DP Beta'] * signals['DP'].shift(1).to_frame() + dp_forecast['Alpha']/12)*100
dp_return = dp_return.rename(columns = {'DP':'Forecasted Return'})
dp_return = pd.DataFrame(dp_return['Forecasted Return'] * gmo_total.loc[:,['SPY']]['SPY'], columns=dp_return.columns, index=dp_return.index)

ep_return = (ep_forecast['EP Beta'] * signals['EP'].shift(1).to_frame() + ep_forecast['Alpha']/12)*100
ep_return = ep_return.rename(columns = {'EP':'Forecasted Return'})
ep_return = pd.DataFrame(ep_return['Forecasted Return'] * gmo_total.loc[:,['SPY']]['SPY'], columns=ep_return.columns, index=ep_return.index)

factors_return = (np.array(signals.shift(1).loc[:,['DP','EP','US10Y']]) @ np.array(forecast.loc[:,['DP Beta','EP Beta','RF Beta']].T)) 
factors_return = (pd.DataFrame(factors_return,columns = ['Forecasted Return'],index= signals.index)) 
factors_return['Forecasted Return'] = ((factors_return['Forecasted Return'] + float(forecast['Alpha']/12)))*100
factors_return = pd.DataFrame(factors_return['Forecasted Return'] *gmo_total.loc[:,['SPY']]['SPY'], columns=factors_return.columns, index=factors_return.index)

In [21]:
strats = {'DP': dp_return.dropna(),
          'EP': ep_return.dropna(),
          'DP-EP-US10Y': factors_return.dropna()
         }
factor = gmo.loc[:,['SPY']]
strat_summary =[]
for k,v in strats.items():
    strat = strats[k]
    perf_summary = performance_summary(strat)
    perf_summary['Negative Risk Premium Months'] = len(strat[strat['Forecasted Return'] - rf['US3M'] <0])
    perf_summary['Total Months'] = len(strat)
    perf_summary.index = [k]
    # X is SPY, y is returns predicted by signals
    reg = regression_based_performance(factor[strat.index[0]:],strat)
    perf_summary['Market Beta'] = reg[0][0]
    perf_summary['Market Alpha'] = reg[3]
    perf_summary['Market Information Ratio'] = reg[2]
    strat_summary.append(perf_summary)
    

strat_summary_df = pd.concat(strat_summary)
strat_summary_df.loc[:,['Mean','Volatility','Sharpe Ratio','Max Drawdown','Market Beta','Market Alpha','Market Information Ratio']]

,Mean,Volatility,Sharpe Ratio,Max Drawdown,Market Beta,Market Alpha,Market Information Ratio
DP,0.109539,0.148858,0.735859,-0.656967,0.861746,0.041112,0.549044
EP,0.108055,0.128905,0.838249,-0.385317,0.733554,0.049800,0.732559
DP-EP-US10Y,0.125094,0.145603,0.859145,-0.524606,0.778129,0.063300,0.721235


### 3.3 Risk Characteristics

#### (a) VAR

In [22]:
market_summary = performance_summary(gmo.loc[:,['SPY']])
gmo_summary = performance_summary(gmo.loc[:,['GMWAX']].dropna())
strat_var= pd.concat([strat_summary_df.loc[:,['VaR (0.05)']],market_summary.loc[:,['VaR (0.05)']],gmo_summary.loc[:,['VaR (0.05)']]])
strat_var

,VaR (0.05)
DP,-0.052335
EP,-0.053892
DP-EP-US10Y,-0.064082
SPY,-0.073525
GMWAX,-0.047061


#### (b) 2000-2011 Beating RF

In [23]:
strats = {'DP': dp_return.dropna(),
          'EP': ep_return.dropna(),
          'DP-EP-US10Y': factors_return.dropna(),
          'Risk Free Rate': rf['US3M'].to_frame('Forecasted Return')
         }
strat_summary_0011 =[]
for k,v in strats.items():
    strat = (strats[k]['2000':'2011']['Forecasted Return']).to_frame('Forecasted Returns')
    perf_summary = performance_summary(strat)
    perf_summary.index = [k]
    strat_summary_0011.append(perf_summary)
    

strat_summary_df_0011 = pd.concat(strat_summary_0011)
strat_summary_df_0011.loc[:,['Mean','Volatility','Sharpe Ratio','Max Drawdown']]

,Mean,Volatility,Sharpe Ratio,Max Drawdown
DP,0.039709,0.186016,0.213468,-0.656967
EP,0.037709,0.134767,0.279805,-0.385317
DP-EP-US10Y,0.061471,0.158851,0.386972,-0.524606
Risk Free Rate,0.023062,0.005785,3.986632,0.000000


The risk-free rate from 2000 - 2011 has a mean of 0.023 while all the dynamic portfolios have higher means than the risk-free rate. They do not underperform the risk-free rate over this time period. 

#### (c) Negative Premium

In [24]:
neg_risk_premium = strat_summary_df.loc[:,['Negative Risk Premium Months','Total Months']]
neg_risk_premium['Negative Risk Premium Months (%)'] = neg_risk_premium['Negative Risk Premium Months'] *100/ neg_risk_premium['Total Months']
neg_risk_premium

,Negative Risk Premium Months,Total Months,Negative Risk Premium Months (%)
DP,139,368,37.771739
EP,139,368,37.771739
DP-EP-US10Y,138,368,37.500000


#### (d) Extra Risk

In [25]:
strat_summary_df.loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)','Max Drawdown','Market Beta','Market Alpha','Market Information Ratio']]

,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown,Market Beta,Market Alpha,Market Information Ratio
DP,0.109539,0.148858,0.735859,-0.052335,-0.656967,0.861746,0.041112,0.549044
EP,0.108055,0.128905,0.838249,-0.053892,-0.385317,0.733554,0.049800,0.732559
DP-EP-US10Y,0.125094,0.145603,0.859145,-0.064082,-0.524606,0.778129,0.063300,0.721235


In [26]:
market_summary.loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)','Max Drawdown']]

,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown
SPY,0.07946,0.149097,0.532939,-0.073525,-0.560012


If we consider the question based on the statistical outcomes above, the VaR of dynamic strategies is smaller than the stock market (SPY) which is at -0.07 and their volatilities are also smaller than SPY which is at 0.15. Although the DP strategy has a worse max drawdown, they are not taking on extra risk than the market. However, the predictions and regressions are made by in-sample data; the performance of these dynamic strategies can perform worse in the out-of-sample period. The inherent risk of prediction error is notable. 

# 4. Out-of-Sample

### 4.1 R-Squared

In [27]:
# function 

def OOS_r2(df, X, window):
    y = df['SPY']
    X = sm.add_constant(X)

    forecast_err, null_err = [], []

    # eumerate: for index,value in eumerate(iterable)
    for i,j in enumerate(df.index):
        if i >= window:
            # for an expanding window
            currX = X.iloc[:i]
            currY = y.iloc[:i]

            # apply OLS on X and Y
            reg = sm.OLS(currY, currX, missing = 'drop').fit()

            null_forecast = currY.mean()

            # regression prediction y value using x value
            reg_predict = reg.predict(X.iloc[[i]])

            actual = y.iloc[[i]]

            forecast_err.append(reg_predict - actual)
            null_err.append(null_forecast - actual)
            
    RSS = (np.array(forecast_err)**2).sum()
    TSS = (np.array(null_err)**2).sum()
    
    return ((1 - RSS/TSS),reg)

In [28]:
factor = signals.loc[:,'EP'].shift(1).to_frame()
fund_ret = gmo_total.loc[factor.index[0]:,['SPY']]
reg_ep = OOS_r2(fund_ret,factor,60)
OOS_RSquared_ep = reg_ep[0]
OOS_r2_ep = pd.DataFrame([[OOS_RSquared_ep]], columns = ['OOS R-Squared'], index = ['EP'])
reg_ep_params = reg_ep[1]

In [29]:
factor = signals.loc[:,'DP'].shift(1).to_frame()
fund_ret = gmo_total.loc[factor.index[0]:,['SPY']]
reg_dp = OOS_r2(fund_ret,factor,60)
OOS_RSquared_dp = reg_dp[0]
OOS_r2_dp = pd.DataFrame([[OOS_RSquared_dp]], columns = ['OOS R-Squared'], index = ['DP'])
reg_dp_params = reg_dp[1]

In [30]:
factor = signals.loc[:,['DP','EP']].shift(1)
fund_ret = gmo_total.loc[factor.index[0]:,['SPY']]
reg_epdp = OOS_r2(fund_ret,factor,60)
OOS_r2_epdp  = reg_epdp[0]
OOS_r2_epdp = pd.DataFrame([[OOS_r2_epdp]], columns = ['OOS R-Squared'], index = ['DP-EP'])
reg_epdp_params = reg_epdp[1]

In [31]:
factor = signals.loc[:,['DP','EP','US10Y']].shift(1)
fund_ret = gmo_total.loc[factor.index[0]:,['SPY']]
reg_all = OOS_r2(fund_ret,factor,60)
OOS_RSquared_all  = reg_all[0]
OOS_r2_all = pd.DataFrame([[OOS_RSquared_all]], columns = ['OOS R-Squared'], index = ['All'])
reg_all_params = reg_all[1]

In [32]:
oos_r2_sum = pd.concat([OOS_r2_dp,OOS_r2_ep,OOS_r2_epdp,OOS_r2_all])
oos_r2_sum

,OOS R-Squared
DP,-0.002074
EP,-0.006394
DP-EP,-0.017227
All,-0.030651


No, all those forecasting strategies do not provide positive OOS R^2; the DP-only model has R^2 at -0.002, the EP-only model has R^2 at -0.006, the EP-DP model has R^2 at -0.017; if the prediction model includes all 3 factors, the R^2 is the worst, at -0.031. In other words, the predictive power of these models is worse than the null forecast using the historical mean to predict the future SPY return; the error becomes larger when more factors are included. 

### 4.2 OOS Forecast

In [33]:
# function

def OOS_strat(df, factors, start, weight):
    returns = []
    y = df['SPY']
    X = sm.add_constant(factors)

    for i,j in enumerate(df.index):
        if i >= start:
            # expanding window
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            pred = reg.predict(X.iloc[[i]])
            w = pred * weight
            returns.append((df.iloc[i]['SPY'] * w)[0])

    df_strat = pd.DataFrame(data = returns, index = df.iloc[-(len(returns)):].index, columns = ['Strat Returns'])
    return df_strat

In [34]:
factor = signals.loc[:,'EP'].shift(1).to_frame()
fund_ret= gmo_total.loc[factor.index[0]:,['SPY']]
OOS_EP_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'EP_OOS_Returns'})

In [35]:
factor = signals.loc[:,'DP'].shift(1).to_frame()
fund_ret= gmo_total.loc[factor.index[0]:,['SPY']]
OOS_DP_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'DP_OOS_Returns'})

In [36]:
factor = signals.loc[:,['DP','EP']].shift(1)
fund_ret= gmo_total.loc[factor.index[0]:,['SPY']]
OOS_EPDP_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'DP-EP_OOS_Returns'})

In [37]:
factor = signals.loc[:,['DP','EP','US10Y']].shift(1)
fund_ret= gmo_total.loc[factor.index[0]:,['SPY']]
OOS_all_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'All_OOS_Returns'})

In [38]:
oos_prediction_sum = pd.concat([OOS_DP_predict.T,OOS_EP_predict.T,OOS_EPDP_predict.T,OOS_all_predict.T])
oos_prediction_sum = oos_prediction_sum.T
oos_prediction_sum

,DP_OOS_Returns,EP_OOS_Returns,DP-EP_OOS_Returns,All_OOS_Returns
1998-02-28,0.164322,0.051217,0.115358,0.112804
1998-03-31,0.137497,0.045644,0.100353,0.100353
1998-04-30,0.039485,0.014350,0.029556,0.029678
1998-05-31,-0.061861,-0.022508,-0.044640,-0.044867
1998-06-30,0.109189,0.035103,0.071699,0.071650
...,...,...,...,...
2023-06-30,0.040872,0.047619,0.036428,0.037490
2023-07-31,0.019252,0.022033,0.015283,0.015221
2023-08-31,-0.009011,-0.010477,-0.006695,-0.006397
2023-09-30,-0.026657,-0.032092,-0.021321,-0.019959


In [39]:
strats = {'DP': OOS_DP_predict.dropna(),
          'EP': OOS_EP_predict.dropna(),
          'DP-EP':OOS_EPDP_predict.dropna(),
          'All': OOS_all_predict.dropna(),
          'SPY':gmo.loc[OOS_all_predict.index[0]:,['SPY']].rename(columns={'SPY':'SPY_OOS_Returns'}),
          'US3M':rf['US3M'].to_frame('US3M_OOS_Returns')
         }
factor = gmo.loc[:,['SPY']]
strat_summary =[]
for k,v in strats.items():
    strat = strats[k]
    perf_summary = performance_summary(strat)
    perf_summary['Negative Risk Premium Months'] = len(strat[strat[k+'_OOS_Returns'] - rf['US3M'] <0])
    perf_summary['Total Months'] = len(strat)
    perf_summary.index = [k]
    reg = regression_based_performance(factor[strat.index[0]:],strat,0)
    perf_summary['Market Beta'] = reg[0][0]
    perf_summary['Market Alpha'] = reg[3]
    perf_summary['Market Information Ratio'] = reg[2]
    strat_summary.append(perf_summary)
    

strat_summary_df = pd.concat(strat_summary)
strat_summary_df.loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)','Max Drawdown','Market Beta','Market Alpha','Market Information Ratio']]

,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown,Market Beta,Market Alpha,Market Information Ratio
DP,0.079626,0.173732,0.458326,-0.071179,-0.551925,0.997390,7.805613e-02,0.801191
EP,0.082373,0.163741,0.503066,-0.068431,-0.583693,0.559981,1.394802e-01,0.283310
DP-EP,0.096815,0.226111,0.428174,-0.071698,-0.760910,0.484359,2.139194e-01,0.111792
All,0.112022,0.247893,0.451897,-0.071882,-0.804959,0.508122,2.358112e-01,0.102183
SPY,0.067239,0.156070,0.430826,-0.080066,-0.560012,1.000000,3.322999e-17,1.000000
US3M,0.023803,0.006165,3.860934,0.000020,0.000000,0.005793,6.249634e-03,0.008948


Compared to the in-sample version, the strategies perform worse in terms of lower mean returns, higher volatilities, lower Sharpe ratios, lower VaR, and worse Max Drawdown (except the DP model). It is obvious that the OOS strategy will be worse compared to the in-sample strategy as the predictive power of the model is not reliable due to low R^2, especially for OOS. 

### 4.3 Risk Characteristics

In [40]:
strat_var= pd.concat([strat_summary_df.loc[:,['VaR (0.05)']],market_summary.loc[:,['VaR (0.05)']],gmo_summary.loc[:,['VaR (0.05)']]])
strat_var

,VaR (0.05)
DP,-0.071179
EP,-0.068431
DP-EP,-0.071698
All,-0.071882
SPY,-0.080066
US3M,0.000020
SPY,-0.073525
GMWAX,-0.047061


Compared with models in section 3, the VaR of OOS strategies is lower; it is taking higher risks in this sense. 

In [41]:
strats = {'DP': OOS_DP_predict.dropna(),
          'EP': OOS_EP_predict.dropna(),
          'DP-EP':OOS_EPDP_predict.dropna(),
          'All': OOS_all_predict.dropna(),
          'US3M':rf['US3M'].to_frame('US3M_OOS_Returns')
         }
factor = gmo.loc[:,['SPY']]['2000':'2011']
strat_summary =[]
for k,v in strats.items():
    strat = strats[k]['2000':'2011']
    perf_summary = performance_summary(strat)
    perf_summary['Negative Risk Premium Months'] = len(strat[strat[k+'_OOS_Returns'] - rf['2000':'2011']['US3M'] <0])
    perf_summary['Total Months'] = len(strat)
    perf_summary.index = [k]
    reg = regression_based_performance(factor[strat.index[0]:],strat)
    perf_summary['Market Beta'] = reg[0][0]
    perf_summary['Market Alpha'] = reg[3]
    perf_summary['Market Information Ratio'] = reg[2]
    strat_summary.append(perf_summary)
    

strat_summary_df_0011 = pd.concat(strat_summary)
strat_summary_df_0011.loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)','Max Drawdown','Market Beta','Market Alpha','Market Information Ratio']]

,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown,Market Beta,Market Alpha,Market Information Ratio
DP,-0.010895,0.163221,-0.066749,-0.094730,-0.551925,0.952248,-0.006228,-0.124928
EP,0.038768,0.195919,0.197877,-0.085329,-0.583693,0.296065,0.040224,0.211832
DP-EP,0.043290,0.290943,0.148793,-0.100111,-0.760910,0.076108,0.043668,0.150212
All,0.084077,0.328892,0.255636,-0.091413,-0.804959,0.114138,0.084636,0.257752
US3M,0.023062,0.005785,3.986632,0.000035,0.000000,-0.002853,0.023052,3.997186


Compared with the risk-free rate (0.02), the dynamic portfolios are performing better except for the DP strategy which has a mean of -0.01; all other strategies outperformed the risk-free rate during the 2000 - 2011 period. However, compared with models in question 3, the performance of OOS strategies have lower mean returns and much worse Sharpe Ratio. 

In [42]:
neg_risk_premium = strat_summary_df.loc[:,['Negative Risk Premium Months','Total Months']]
neg_risk_premium['Negative Risk Premium Months (%)'] = neg_risk_premium['Negative Risk Premium Months'] *100/ neg_risk_premium['Total Months']
neg_risk_premium

,Negative Risk Premium Months,Total Months,Negative Risk Premium Months (%)
DP,122,309,39.482201
EP,120,309,38.834951
DP-EP,121,309,39.158576
All,117,309,37.864078
SPY,125,309,40.453074
US3M,0,369,0.000000


Compared with models in section 3, the OOS strategies have a higher probability of seeing negative risk premiums. It is taking extra risks. 

In [43]:
pd.concat([strat_summary_df, market_summary]).loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)','Max Drawdown']]

,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown
DP,0.079626,0.173732,0.458326,-0.071179,-0.551925
EP,0.082373,0.163741,0.503066,-0.068431,-0.583693
DP-EP,0.096815,0.226111,0.428174,-0.071698,-0.760910
All,0.112022,0.247893,0.451897,-0.071882,-0.804959
SPY,0.067239,0.156070,0.430826,-0.080066,-0.560012
US3M,0.023803,0.006165,3.860934,0.000020,0.000000
SPY,0.079460,0.149097,0.532939,-0.073525,-0.560012


The volatilities of OOS strategies are higher than the market volatilities and show lower Sharpe Ratios. The VaR is slightly better while the Max Drawdown still is larger than the SPY; aligned with the previous analysis compared with the in-sample dynamic strategy, we conclude that OOS strategies are taking on extra risks and the point-in-time version of the strategy will be riskier. 

---